# **DataFrame imdb**

## **Import des bibliothèques nécessaires**

In [50]:
import pandas as pd

## **Chargement des datasets IMDb**

In [ ]:
# Nous importons les données directement depuis les fichiers IMDb compressés au format `.tsv.gz`.

#* Datasets principaux
df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

#* Datasets complémentaires
df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

C:\Users\koke7\AppData\Local\Temp\ipykernel_15176\2730036939.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')
C:\Users\koke7\AppData\Local\Temp\ipykernel_15176\2730036939.py:10: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep = '\t', compression='gzip', na_values='\\N')


## **Nettoyage des datasets IMDb**

In [ ]:

#! === SETUP ===
#? title_basics

# 1. Garder uniquement les films (`titleType = 'movie'`).
# 2. Supprimer les lignes avec des valeurs manquantes essentielles.
# 3. Filtrer les années de production pour garder uniquement celles entre 1970 et 2025.
# 4. Réduire le dataset aux colonnes essentielles.

df_title_basics_clean = (
    df_title_basics[df_title_basics['titleType'] == 'movie']
    .dropna(subset=['startYear', 'genres'])
    .assign(startYear=lambda x: x['startYear'].astype(int))
    .query('1970 <= startYear <= 2025')
    [['tconst', 'primaryTitle']]
)


#! === SETUP ===
#? title_rating

# 1. Garder uniquement les films ayant plus de 1000 votes.

df_title_ratings_clean = df_title_ratings[df_title_ratings['numVotes'] > 1000]


#! === SETUP ===
#? title_akas

# 1. Garder uniquement les titres traduits en français (`region = 'FR'`).
# 2. Réduire aux colonnes essentielles et renommer pour correspondre au format attendu.

df_title_akas_clean = (
    df_title_akas[df_title_akas['region'] == 'FR']
    [['titleId', 'title']]
    .rename(columns={'titleId': 'tconst', 'title': 'Titre Français'})
)


#! === SETUP ===
#? title_crew

# 1. Remplir les valeurs manquantes dans `directors` avec "Unknown".
# 2. Transformer `directors` en une liste.
# 3. Exploser les réalisateurs en lignes individuelles pour fusionner avec `name.basics`.
# 4. Regrouper les réalisateurs par film sous forme de liste.

df_title_crew_clean = (
    df_title_crew.fillna({'directors': 'Unknown'})
    .assign(directors=lambda x: x['directors'].str.split(','))
    .explode('directors')
    .merge(df_name_basics[['nconst', 'primaryName']], left_on='directors', right_on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)


#! === SETUP ===
#? title_principals

# 1. Filtrer les acteurs uniquement.
# 2. Fusionner avec `name.basics` pour récupérer les noms des acteurs.
# 3. Regrouper les acteurs par film sous forme de liste.

df_title_principals_clean = (
    df_title_principals.dropna(subset=['category'])
    .query("category == 'actor'")
    .merge(df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)

## **Fusion des datasets IMDb**

In [ ]:

#! === MERGEUP ===
#* Nous fusionnons les datasets nettoyés pour créer un seul dataframe regroupant toutes les informations utiles.

#* Fusion `title.basics` et `title.ratings`
df_merged_v1 = df_title_basics_clean.merge(df_title_ratings_clean, on='tconst')

#* Fusion avec `title.akas`
df_merged_v2 = df_merged_v1.merge(df_title_akas_clean, on='tconst', how='left')

#* Fusion avec `title.crew`
df_merged_v3 = df_merged_v2.merge(df_title_crew_clean, on='tconst', how='left')

#* Ajout des acteurs depuis `title.principals`
df_merged_v3['Acteurs'] = df_merged_v3['tconst'].map(dict(zip(df_title_principals_clean['tconst'], df_title_principals_clean['primaryName'])))

#* Export du dataframe IMDb final
df_merged_v3.to_csv("../data/raw/df_movie.csv", index=False)

# **Initialisation de TMDB**

## **Chargement des données**

In [52]:

#! === IMPORT ===
df_tmdb = pd.read_csv('../data/raw/tmdb_full.csv')
df_movie = pd.read_csv('../data/raw/df_movie.csv')


C:\Users\koke7\AppData\Local\Temp\ipykernel_30340\1125499810.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmdb = pd.read_csv('../data/raw/tmdb_full.csv')


## **Initialisation des datasets**

In [53]:

#* Nous conservons uniquement les colonnes pertinentes pour le projet.
df_tmdb = df_tmdb[
    ['backdrop_path', 'budget', 'genres', 'imdb_id', 'original_language', 'popularity', 
     'poster_path', 'production_countries', 'release_date', 'revenue', 'runtime', 
     'spoken_languages', 'vote_average', 'vote_count', 'production_companies_name']
]

#* Renommage des colonnes pour le dataset IMDb (df_movie)
df_movie.rename(columns={
    'tconst': 'ID',
    'primaryTitle': 'Titre Original',
    'averageRating': 'Note imdb',
    'numVotes': 'Votes imdb',
    'title': 'Titre Français',
    'actors': 'Acteurs',
    'primaryName': 'Réalisateur(s)'
}, inplace=True)

#* Mise en forme des colonnes "Réalisateur(s)" et "Acteurs" pour convertir les listes en chaînes lisibles.
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].str.strip("[]").str.replace("'", "")

df_movie['Acteurs'] = df_movie['Acteurs'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Acteurs'] = df_movie['Acteurs'].str.strip("[]").str.replace("'", "")

#* Renommage des colonnes pour le dataset TMDB (df_tmdb)
df_tmdb.rename(columns={
    'backdrop_path': 'Image de Fond',
    'budget': 'Budget',
    'genres': 'Genres',
    'imdb_id': 'ID IMDb',
    'original_language': 'Langue Originale',
    'popularity': 'Popularité',
    'poster_path': 'Affiche',
    'production_countries': 'Pays de Production',
    'release_date': 'Date de Sortie',
    'revenue': 'Box Office',
    'runtime': 'Durée',
    'spoken_languages': 'Langues Parlées',
    'vote_average': 'Note tmdb',
    'vote_count': 'Votes tmdb',
    'production_companies_name': 'Compagnies de Production'
}, inplace=True)

#* Mise en forme des colonnes avec des listes pour les rendre lisibles.
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].str.strip("[]").str.replace("'", "")

df_tmdb['Genres'] = df_tmdb['Genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Genres'] = df_tmdb['Genres'].str.strip("[]").str.replace("'", "")

## **Fusion de Imdb et Tmdb**

In [54]:

#! === MERGEUP ===

df_final = pd.merge(df_movie, df_tmdb, left_on='ID', right_on='ID IMDb')

## **Filtrage Général**

In [56]:

#* Critères de sélection pour inclure uniquement les films pertinents dans l'analyse.
df_movie_cleaned_v1 = df_final[
    # Films en français avec plus de 5000 votes et popularité > 2.5
    ((df_final['Votes imdb'] > 5000) & (df_final['Popularité'] > 2.5) & (df_final['Langue Originale'] == 'fr')) |
    # Films avec des langues parlées incluant le français, plus de 15000 votes et popularité > 7.5
    ((df_final['Votes imdb'] > 15000) & (df_final['Popularité'] > 7.5) & (df_final['Langues Parlées'].apply(lambda x: 'fr' in x))) |
    # Films en anglais avec plus de 30000 votes et popularité > 15
    ((df_final['Votes imdb'] > 30000) & (df_final['Popularité'] > 15) & (df_final['Langue Originale'] == 'en')) |
    # Films avec au moins 50000 votes et popularité > 25
    (df_final['Votes imdb'] > 50000) & (df_final['Popularité'] > 25)
]


#* On conserve uniquement les films ayant des notes IMDb et TMDb supérieures à 4.
df_movie_cleaned_v2 = df_movie_cleaned_v1[
    (df_movie_cleaned_v1['Note imdb'] > 4) & (df_movie_cleaned_v1['Note tmdb'] > 4)
]


#* Exclusion des films ayant une note IMDb inférieure à 8 et sortis avant l'année 2000.
df_movie_cleaned_v3 = df_movie_cleaned_v2[
    ~((df_movie_cleaned_v2['Note imdb'] < 8) & (df_movie_cleaned_v2['Date de Sortie'] < '2000-01-01'))
]


#* On élimine les doublons en se basant sur le titre original.
df_movie_cleaned_v4 = df_movie_cleaned_v3.drop_duplicates(subset='Titre Original')

#* Création d'une copie explicite pour éviter des avertissements potentiels de pandas.
df_movie_cleaned_v4 = df_movie_cleaned_v4.copy()

#* Conversion de la colonne 'Durée' en format numérique.
df_movie_cleaned_v4['Durée'] = pd.to_numeric(df_movie_cleaned_v4['Durée'], errors='coerce')

## **Ajout de données supplémentaires**

In [57]:

#* Cette colonne catégorise les films en fonction de leur nombre de votes IMDb.
def categorize_votes(votes):
    if votes < 50000:
        return "Connu"
    elif 50000 <= votes < 200000:
        return "Populaire"
    elif 200000 <= votes < 1000000:
        return "Très populaire"
    else:
        return "Blockbuster"

df_movie_cleaned_v4['Réputation'] = df_movie_cleaned_v4['Votes imdb'].apply(categorize_votes)


#* Cette colonne catégorise les films selon leur durée en minutes.
def categorize_times(duree):
    if duree < 100:
        return "Court"
    elif 100 <= duree <= 200:
        return "Moyen"
    else:
        return "Long"

df_movie_cleaned_v4['Métrage'] = df_movie_cleaned_v4['Durée'].apply(categorize_times)


#* Cette colonne regroupe les films par décennie de sortie.
def categorize_years(year):
    if 1970 <= year <= 1979:
        return "70°s"
    if 1980 <= year <= 1989:
        return "80°s"
    if 1990 <= year <= 1999:
        return "90°s"
    if 2000 <= year <= 2009:
        return "2000"
    if 2010 <= year <= 2019:
        return "2010"
    if 2020 <= year <= 2029:
        return "2020"

df_movie_cleaned_v4['Décennie'] = df_movie_cleaned_v4['Date de Sortie'].apply(
    lambda x: categorize_years(int(str(x)[:4])) if isinstance(x, str) else x
)


#* Cette colonne extrait le premier genre de la liste des genres.
df_movie_cleaned_v4['Genre Principal'] = df_movie_cleaned_v4['Genres'].apply(
    lambda x: x.split(',')[0] if isinstance(x, str) else x
)


#* On exclut les films dont le genre principal est 'Documentary'.
df_movie_cleaned_v4 = df_movie_cleaned_v4[df_movie_cleaned_v4['Genre Principal'] != 'Documentary']


#* On repositionne les colonnes pour une meilleure lisibilité et cohérence.
df_movie_cleaned_v4 = df_movie_cleaned_v4[
    ['ID', 'Titre Original', 'Titre Français', 'Réalisateur(s)', 'Acteurs',
     'Budget', 'Genres', 'Genre Principal', 'Date de Sortie', 'Décennie',
     'Langue Originale', 'Langues Parlées', 'Popularité', 'Réputation',
     'Affiche', 'Image de Fond', 'Pays de Production', 'Durée', 'Métrage',
     'Note tmdb', 'Votes tmdb', 'Note imdb', 'Votes imdb',
     'Compagnies de Production', 'Box Office']
]


## **Exportation**

In [58]:

#! === EXPORT ===

df_movie_cleaned_v4.to_csv("../data/processed/df_movie_cleaned.csv", index=False)

## **Rendu :**

In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/df_movie_cleaned.csv")

In [3]:
df

,ID,Titre,averageRating,Nombre de Votes,Titre en Français,Réalisateur,Acteurs,Image de Fond,Budget,Genres,...,Popularité,Affiche,Pays de Production,Date de Sortie,Box Office,Durée,Langues Parlées,Note tmdb,Votes tmdb,Compagnies de Production
0,tt0035423,Kate & Leopold,6.4,90717,Kate et Léopold,['James Mangold'],"['Hugh Jackman', 'Liev Schreiber', 'Breckin Me...",/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,48000000,"['Romance', 'Fantasy', 'Comedy']",...,15.770,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg,['US'],2001-12-25,76019048,118,"['en', 'fr', 'it']",6.326,1187,"['Konrad Pictures', 'Miramax']"
1,tt0038687,Let There Be Light,7.4,2063,Que la lumière soit,['John Huston'],['Walter Huston'],/dJxxKv6eGBNRGVGlMEuO3J8EoNC.jpg,0,"['Documentary', 'War']",...,3.575,/wgcAMb5BLKFANzTDfKwnxeH1kYA.jpg,['US'],1946-12-16,0,58,['en'],7.400,35,['U.S. Army Pictorial Services']
2,tt0054724,I Eat Your Skin,3.6,1718,NaN,['Del Tenney'],"['William Joyce', 'Walter Coy', 'Dan Stapleton...",/ygOSnjgigJn9D42Yt3ZNyBwwK2s.jpg,0,['Horror'],...,2.358,/mbBgHCs9gR6ba3oQ24FH0TZ2vbl.jpg,['US'],1971-10-29,0,84,['en'],3.353,17,['Iselin-Tenney Productions Inc.']
3,tt0061592,Doomsday Machine,2.6,1421,NaN,"['Harry Hope', 'Lee Sholem', 'Herbert J. Leder']","['Bobby Van', 'James Craig', 'Grant Williams',...",/m3EKZKunYksXMNU11lxd8dYxcmy.jpg,0,['Science Fiction'],...,4.134,/jOgXisaiXRcr1BUM2wycdlRTMG7.jpg,['US'],1972-01-01,0,83,['en'],3.068,22,['First Leisure']
4,tt0062690,The Awakening of the Beast,5.9,1340,L'éveil de la bête,['José Mojica Marins'],"['Sérgio Hingst', 'Ozualdo Ribeiro Candeias', ...",/s5X09LpwG70jr5md52eZNwzHUtU.jpg,0,"['Horror', 'Drama']",...,1.650,/u2klRQMwdMpv8NtD0qoy2TjByv1.jpg,['BR'],1970-01-01,0,93,['pt'],5.196,23,"['Ovni Indústria Cinematográfica', 'Fotocena F..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39844,tt9907782,The Cursed,6.2,19035,The Cursed,['Sean Ellis'],"['Boyd Holbrook', 'Alistair Petrie', 'Nigel Be...",/gRTYArznK1MLAq7opC1tYHnTwM3.jpg,0,"['Horror', 'Fantasy', 'Mystery']",...,22.872,/bewmBcjJxHeipS1PNu9Lfyvt3zY.jpg,['US'],2021-07-28,4588389,111,['en'],6.900,169,"['Piste Rouge', 'LD Entertainment']"
39845,tt9908390,Le lion,5.5,1460,Le lion,['Ludovic Colbeau-Justin'],"['Dany Boon', 'Philippe Katerine', 'Samuel Jou...",/5nq95dRJCltmrA2PJniSW5mSfWh.jpg,0,['Comedy'],...,7.731,/zfLuACsPcwH6aonTKukn96d1qpp.jpg,['FR'],2020-01-29,0,95,['fr'],5.328,172,"['TF1 Studio', 'Monkey Pack Films', 'Pathé', '..."
39846,tt9911196,The Marriage Escape,7.4,3394,De Beentjes van Sint-Hildegard,['Johan Nijenhuis'],"['Herman Finkers', 'Ferdi Stofmeel', 'Stef Ass...",/f1WhKsQZEPjOHQ4wTmlq75z0RF6.jpg,0,"['Comedy', 'Drama']",...,3.420,/qZNlF6i3MOvwhRV8SDfp5M40hVx.jpg,['NL'],2020-02-10,0,103,['nl'],7.800,31,"['Johan Nijenhuis & Co', 'Omroep MAX']"
39847,tt9916270,Il talento del calabrone,5.8,1494,NaN,['Giacomo Cimini'],"['Sergio Castellitto', 'Lorenzo Richelmy', 'Da...",/w3eAsDs1HXWsz4GgcICR8rR4Mdm.jpg,0,['Thriller'],...,6.788,/5pAAjVDpq4cCCsZsdivkCLMY6Hm.jpg,['IT'],2020-11-18,0,93,['it'],6.388,410,"['PACO Cinematografica', 'Eagle Pictures']"
